In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers

In [ ]:
from diffusers import DiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import os
import json
import random
import shutil
import matplotlib.pyplot as plt

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_clip_score(image, prompt):
    inputs = clip_processor(
        text=[prompt[:75]],
        images=image,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to("cuda")

    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    score = logits_per_image.softmax(dim=1)[0][0].item()
    return score

In [ ]:
genders = ['male', 'female']

emotion_prompts = {
    'happy': 'smiling, joyful, delighted',
    'sad': 'frowning, sad face expression, crying',
    'angry': 'angry, hostility',
    'surprised': 'surprised, opened mouth, raised eyebrows'
}

with open('ethnicities.json', 'r') as f:
    ethnicities = json.load(f)

In [ ]:
positive_prompt = 'Medium-shot portrait of {} {}, {}, human, front view, looking at the camera, sharp clear detailed expressive eyes, perfect face, symmetrical face, color photography, photorealistic, hyperrealistic, realistic, incredibly detailed, crisp focus, digital art, depth of field, 50mm, 8k'
negative_prompt = '3d, cartoon, anime, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), Low Quality, Worst Quality, plastic, fake, disfigured, fused eyes, mutated face, deformed, blurry, bad anatomy, blurred, watermark, grainy, signature'


In [ ]:




for j in range(3):
    for emotion, emotion_prompt in emotion_prompts.items():
        ethnicity = random.choice(ethnicities)
        gender = random.choice(genders)
        prompt = positive_prompt.format(ethnicity, gender, emotion_prompt)

        best_img, best_score = None, -1
        for _ in range(3):
            image = pipeline(prompt, negative_prompt=negative_prompt).images[0]
            score = get_clip_score(image, prompt)

            if score > best_score:
                best_score = score
                best_img = image

        save_path = os.path.join(BASE_DIR, emotion, f"{str(j).zfill(4)}.png")
        best_img.save(save_path)



In [ ]:
shutil.make_archive("faces_dataset", 'zip', BASE_DIR)
print("All images saved and zipped as 'faces_dataset.zip'")